# Converters

Converters are used to transform prompts before sending them to the target.

This can be useful for a variety of reasons, such as encoding the prompt in a different format, or adding additional information to the prompt. For example, you might want to convert a prompt to base64 before sending it to the target, or add a prefix to the prompt to indicate that it is a question.

## Simple Converter Example

Converters can be used to perform these types of transformations. Here is a simple program that uses Rot13Converter converter, RandomCapitalLettersConverter, and AsciiArtConverter.

In [1]:
from pyrit.common import default_values
from pyrit.prompt_converter import ROT13Converter, AsciiArtConverter, RandomCapitalLettersConverter


default_values.load_environment_files()

prompt = "tell me how to cut down a tree"


print(await ROT13Converter().convert_tokens_async(prompt=prompt))  # type: ignore
print(await RandomCapitalLettersConverter(percentage=25.0).convert_tokens_async(prompt=prompt))  # type: ignore
print(await AsciiArtConverter().convert_tokens_async(prompt=prompt))  # type: ignore

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


text: gryy zr ubj gb phg qbja n gerr
text: tell mE how to cut DoWn A trEe
text: ######   #######  ####     ####              ##   ##  #######           ##   ##   #####   ##   ##           ######    #####              ####   ##   ##  ######            #####     #####   ##   ##  ##   ##             ###             ######   ######   #######  #######  
  ##      ##       ##       ##               ### ###   ##               ##   ##  ##   ##  ##   ##             ##     ##   ##            ##  ##  ##   ##    ##               ## ##   ##   ##  ##   ##  ###  ##            ## ##              ##     ##   ##   ##       ##      
  ##      ##       ##       ##               #######   ##               ##   ##  ##   ##  ##   ##             ##     ##   ##           ##       ##   ##    ##               ##  ##  ##   ##  ##   ##  #### ##           ##   ##             ##     ##   ##   ##       ##      
  ##      ####     ##       ##               ## # ##   ####             #######  ##   ##  ## # ##          